In [3]:
import gymnasium as gym
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.evaluation import evaluate_policy
import optuna

In [4]:
def make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight):
    """
    Crea e restituisce l'ambiente HalfCheetah-v5 dalla libreria Gymnasium con i parametri specificati.
    """
    return gym.make("HalfCheetah-v5", 
                    reset_noise_scale=reset_noise_scale, 
                    forward_reward_weight=forward_reward_weight, 
                    ctrl_cost_weight=ctrl_cost_weight)

# Hyperparameter tuning con Optuna per SAC
def objective(trial):
    # Parametri dell'environment
    reset_noise_scale = trial.suggest_float('reset_noise_scale', 0.005, 0.3)  
    forward_reward_weight = trial.suggest_float('forward_reward_weight', 0.7, 1.5)
    ctrl_cost_weight = trial.suggest_float('ctrl_cost_weight', 0.02, 0.3)

    # Crea l'ambiente
    env = make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

    # Iperparametri per il modello SAC
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
    buffer_size = trial.suggest_categorical('buffer_size', [100000, 300000, 500000])
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256, 512])
    tau = trial.suggest_float('tau', 0.005, 0.02)
    gamma = trial.suggest_float('gamma', 0.95, 0.9999)

    # Per l’entropia, possiamo scegliere tra un coefficiente fisso oppure "auto"
    # Se si desidera variare su diversi valori, possiamo usare una scelta categoriale
    ent_coef = trial.suggest_categorical('ent_coef', ['auto', 0.001, 0.01, 0.05])

    # Frequenza di training e gradient steps
    train_freq = trial.suggest_categorical('train_freq', [1, 64, 256, 512])
    gradient_steps = trial.suggest_categorical('gradient_steps', [1, 8, 16, 32])

    # Crea ed allena il modello SAC
    model = SAC(
        "MlpPolicy",
        env,
        learning_rate=learning_rate,
        buffer_size=buffer_size,
        batch_size=batch_size,
        tau=tau,
        gamma=gamma,
        train_freq=train_freq,          # Quante azioni eseguite prima di un update
        gradient_steps=gradient_steps,  # Numero di passate di gradienti dopo ogni train_freq
        ent_coef=ent_coef,
        verbose=0,
    )

    model.learn(total_timesteps=50_000)

    # Disabilita gli aggiornamenti di normalizzazione per la valutazione
    env.training = False
    env.norm_reward = False

    # Valuta il modello su 100 episodi in modo deterministico
    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=100, deterministic=True)

    print(f'Mean reward: {mean_reward}')
    return mean_reward

# Crea lo studio Optuna e ottimizza l'obiettivo
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Stampa i migliori iperparametri trovati
print("Best hyperparameters:", study.best_params)


[I 2025-02-23 11:24:46,254] A new study created in memory with name: no-name-b33fca77-e850-4764-a581-1497a2aa5a6c
/var/folders/d5/8f8lxk3x1gddrb19zv8xl57m0000gn/T/ipykernel_61413/3614903701.py:23: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
